In [126]:
import os
import gpxpy
import requests
import numpy as np
import pandas as pd
import sqlite3 as sq
import seaborn as sb
import matplotlib.pyplot as plt

from geopy.distance import geodesic
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
data = pd.read_csv("datasets/gpx-tracks-from-hikr.org.csv")

In [120]:
data

,_id,length_3d,user,start_time,max_elevation,bounds,uphill,moving_time,end_time,max_speed,gpx,difficulty,min_elevation,url,downhill,name,length_2d,parsed_gpx
0,5afb229e8f80884aaad9c6ea,10832.953016,Bergfritz,2018-05-11 07:37:40,1934.470000,"{'min': {'type': 'Point', 'coordinates': [13.2...",612.880000,12155.0,2018-05-11 11:38:23,1.595493,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,1322.960000,http://www.hikr.org/tour/post131855.html,609.670000,"Remsteinkopf, 1945 m",10832.953016,"[{'latitude': 47.231143, 'longitude': 13.22700..."
1,5afb229e8f80884aaad9c6eb,12259.376315,Bergfritz,2018-05-12 07:25:08,2186.210000,"{'min': {'type': 'Point', 'coordinates': [13.1...",614.753000,13876.0,2018-05-12 12:08:28,1.394320,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T3 - Difficult Mountain hike,1266.400000,http://www.hikr.org/tour/post131856.html,1193.733000,"Schuhflicker, 2214 m",12259.376315,"[{'latitude': 47.228474, 'longitude': 13.16851..."
2,5afb229e8f80884aaad9c6ec,22980.168081,igor,2018-05-11 06:29:38,2265.000000,"{'min': {'type': 'Point', 'coordinates': [8.99...",2255.976000,28971.0,2018-05-11 15:32:43,1.503002,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T3 - Difficult Mountain hike,176.540000,http://www.hikr.org/tour/post131839.html,2177.626000,Cima d'erbea Est quota 2164m e Gaggio 2267m,22980.168081,"[{'latitude': 46.18001941591501, 'longitude': ..."
3,5afb229e8f80884aaad9c6ed,24903.503470,rkroebl,2018-05-10 07:06:22,962.420000,"{'min': {'type': 'Point', 'coordinates': [8.43...",882.312000,26726.0,2018-05-10 16:15:18,1.516689,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,388.510000,http://www.hikr.org/tour/post131840.html,901.052000,Waldstätterweg: Alpnachstad - Buochs,24903.503470,"[{'latitude': 46.95516239851713, 'longitude': ..."
4,5afb229e8f80884aaad9c6ee,19581.273819,rkroebl,2018-05-11 05:44:58,697.570000,"{'min': {'type': 'Point', 'coordinates': [8.61...",310.662000,18197.0,2018-05-11 12:54:25,1.542405,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,438.500000,http://www.hikr.org/tour/post131845.html,305.372000,Waldstätterweg: Buochs - Beckenried und Gersau...,19581.273819,"[{'latitude': 46.97962900623679, 'longitude': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12136,5afb26108f80884aaad9f652,3882.088056,guenter,2013-09-14 10:33:16,1603.135742,"{'min': {'type': 'Point', 'coordinates': [15.5...",672.730664,4306.0,2013-09-14 12:53:09,1.271990,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T3 - Difficult Mountain hike,936.461426,http://www.hikr.org/tour/post70727.html,7.498242,Hohes Waxenegg,3882.088056,"[{'latitude': 47.74823, 'longitude': 15.546882..."
12137,5afb26108f80884aaad9f653,13144.015870,Bergfritz,2013-09-14 05:29:24,1664.660000,"{'min': {'type': 'Point', 'coordinates': [15.2...",928.770000,14660.0,2013-09-14 09:57:36,1.727332,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,826.870000,http://www.hikr.org/tour/post70731.html,925.880000,"Mugel 1630 m , Roßeck 1664 m",13144.015870,"[{'latitude': 47.3508706223, 'longitude': 15.1..."
12138,5afb26108f80884aaad9f654,8865.050380,mountainrescue,2013-09-15 06:43:39,1658.300000,"{'min': {'type': 'Point', 'coordinates': [15.2...",821.240000,8239.0,2013-09-15 09:21:43,2.106299,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T1 - Valley hike,882.600000,http://www.hikr.org/tour/post70744.html,839.040000,Man soll sich nie auf die Meteofuzzis verlasse...,8865.050380,"[{'latitude': 47.377195, 'longitude': 15.18882..."
12139,5afb26108f80884aaad9f655,17047.654835,Curi,2013-09-07 08:19:07,646.620000,"{'min': {'type': 'Point', 'coordinates': [11.6...",307.855000,18354.0,2013-09-07 15:11:42,1.425461,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T1 - Valley hike,425.040000,http://www.hikr.org/tour/post70500.html,289.595000,"Frankenwald: Der ""Mühlenweg""",17047.654835,"[{'latitude': 50.223632, 

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12141 entries, 0 to 12140
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id            12141 non-null  object 
 1   length_3d      12141 non-null  float64
 2   user           12141 non-null  object 
 3   start_time     9897 non-null   object 
 4   max_elevation  10563 non-null  float64
 5   bounds         12141 non-null  object 
 6   uphill         12141 non-null  float64
 7   moving_time    12141 non-null  float64
 8   end_time       9897 non-null   object 
 9   max_speed      12141 non-null  float64
 10  gpx            12141 non-null  object 
 11  difficulty     12141 non-null  object 
 12  min_elevation  10563 non-null  float64
 13  url            12141 non-null  object 
 14  downhill       12141 non-null  float64
 15  name           12141 non-null  object 
 16  length_2d      12141 non-null  float64
dtypes: float64(8), object(9)
memory usage: 1.6+ MB


In [13]:
data.isnull().sum()

_id                 0
length_3d           0
user                0
start_time       2244
max_elevation    1578
bounds              0
uphill              0
moving_time         0
end_time         2244
max_speed           0
gpx                 0
difficulty          0
min_elevation    1578
url                 0
downhill            0
name                0
length_2d           0
parsed_gpx        529
dtype: int64

In [54]:
def parse_gpx(gpx_string, track_id):
    try:
        gpx = gpxpy.parse(gpx_string)
    except Exception as e:
        print(e)
        return None
    points = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append({
                            'track_id': track_id,
                            'latitude': point.latitude,
                            'longitude': point.longitude,
                            'elevation': point.elevation,
                            'time': point.time,
                            'name': track.name
                        })
    return points

In [55]:
all_points = []
trace_id = []
for idx, row in data.iterrows():
    gpx = row['gpx']
    if pd.notna(gpx):
        parsed_gpx = parse_gpx(gpx, idx)
        if parsed_gpx:
            all_points.extend(parsed_gpx)
    if idx == 20:
        break
        
parsed_gpx = pd.DataFrame(all_points)

In [125]:
parsed_gpx.isnull().sum()

track_id            0
latitude            0
longitude           0
elevation           0
time              344
name              593
length_3d           0
time_delta        344
step_frequency    344
dtype: int64

In [105]:
length_3d = []
for track in parsed_gpx.groupby('track_id'):
    track = track[1].sort_values('time')
    length_track = 0
    for idx in track.index[:-1]:
        p1 = (track['latitude'][idx], track['longitude'][idx])
        p2 = (track['latitude'][idx + 1], track['longitude'][idx + 1])
        length_2d = geodesic(p1, p2).meters
        length_track += np.sqrt((length_2d ** 2) + (track['elevation'][idx] - track['elevation'][idx + 1]) ** 2)
    length_3d.append(length_track)

parsed_gpx['length_3d'] = parsed_gpx['track_id'].map(lambda x: length_3d[x])

In [112]:
time_delta = list(parsed_gpx.groupby('track_id')['time'].max() - parsed_gpx.groupby('track_id')['time'].min())

In [114]:
parsed_gpx['time_delta'] = parsed_gpx['track_id'].map(lambda x: time_delta[x])

In [121]:
parsed_gpx['step_frequency'] = parsed_gpx.apply(lambda parsed_gpx: (parsed_gpx['length_3d'] / 0.75) / parsed_gpx['time_delta'].total_seconds(), axis=1)

In [119]:
type(parsed_gpx['time_delta'][2])

pandas._libs.tslibs.timedeltas.Timedelta

In [123]:
parsed_gpx

,track_id,latitude,longitude,elevation,time,name,length_3d,time_delta,step_frequency
0,0,47.231143,13.227007,1322.960000,2018-05-11 07:37:40+00:00,remsteinkopf,10855.704940,0 days 04:00:43,1.002165
1,0,47.230543,13.227488,1330.890000,2018-05-11 07:39:44+00:00,remsteinkopf,10855.704940,0 days 04:00:43,1.002165
2,0,47.230637,13.226510,1344.410000,2018-05-11 07:41:17+00:00,remsteinkopf,10855.704940,0 days 04:00:43,1.002165
3,0,47.231071,13.225765,1350.430000,2018-05-11 07:42:36+00:00,remsteinkopf,10855.704940,0 days 04:00:43,1.002165
4,0,47.231073,13.225428,1350.180000,2018-05-11 07:42:58+00:00,remsteinkopf,10855.704940,0 days 04:00:43,1.002165
...,...,...,...,...,...,...,...,...,...
3152,20,46.135693,8.428355,860.480042,2018-05-06 13:43:36+00:00,None,14566.689457,0 days 07:32:30,0.715368
3153,20,46.136034,8.428848,852.600037,2018-05-06 13:44:38+00:00,None,14566.689457,0 days 07:32:30,0.715368
3154,20,46.135984,8.428438,867.650024,2018-05-06 14:01:32+00:00,None,14566.689457,0 days 07:32:30,0.715368
3155,20,46.136069,8.428600,864.160034,2018-05-06 14:02:52+00:00,None,14566.689457,0 days 07:32:30,0.715368


In [171]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2025-11-03",
	"end_date": "2025-11-03",
	"daily": "temperature_2m_mean",
}
response = requests.get(url, params=params)

In [174]:
print(response.json()['daily']['temperature_2m_mean'])

[9.7]


In [206]:
def mean_day_temp(lat, lon, time):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
    	"latitude": lat,
    	"longitude": lon,
    	"start_date": time.isoformat(),
    	"end_date": time.isoformat(),
    	"daily": "temperature_2m_mean",
    }
    response = requests.get(url, params=params)
    print(response.json())
    return response.json()['daily']['temperature_2m_mean']

In [207]:
temp = []
for track in parsed_gpx.groupby('track_id'):
    track = track[1]
    temp.append(mean_day_temp(track['latitude'][0], track['longitude'][0], track['time'][0]))

{'error': True, 'reason': "Invalid date format. Make sure to use 'YYYY-MM-DD'"}


KeyError: 'daily'

In [185]:
type(track['time'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [189]:
response.json()['daily']

{'time': ['2025-11-03'], 'temperature_2m_mean': [9.7]}

In [205]:
parsed_gpx['time'][0].isoformat()

'2018-05-11T07:37:40+00:00'